# 파이썬으로 데이터 수집하기
데이터를 수집하고, 전처리하고, 변환하는 데 시간을 많이 보낸다. 이 장에서는 다양한 방식을 통해 데이터를 수집하고, 올바른 포멧으로 파이썬에 전달하는 방법을 배워보자.

## 9.1 stdin과 stdout
커맨드라인으로 파이썬 코드를 실행시킨다면 sys.stdin과 sys.stdout으로 데이터를 파이핑(piping)할 수 있다. \
예를 들어 문서를 읽고 주어진 정규표현식(regex)과 매칭되는 줄을 출력해 주는 코드가 있다고 해보자.

In [2]:
# egrep.py
import sys, re
# sys.argv는 커맨드라인에서 사용할 수 있느 모든 인자에 대한 리스트이다.
# sys.argv[0]는 프로그램의 이름을 나타낸다.
# sys.argv[1]는 커맨드라인에서 주어지는 정규표현식이다.
regex = sys.argv[1]

for line in sys.stdin:
    # regex에 매칭된다면 stdout으로 출력한다.
    if re.search(regex, line):
        sys.stdout.write(line)

In [3]:
# 줄의 개수를 세고 출력해 주는 코드를 작성해 보자
# line_count.py
import sys

count = 0
for line in sys.stdin:
    count += 1

# 출력값은 sys.stdout으로 보낸다.
print(count)

0


In [5]:
# p .122장 데이터 확인, 하고 코드 추가

## 9.2 파일 읽기
파일은 코드에서 바로 읽고 쓸 수 있다.

## 9.2.1 텍스트 파일의 기본
텍스트 파일을 작업하기 위해서는 가장 먼저 open으로 파일 객체를 불러와야 한다.

In [7]:
# 'r'은 read-only(읽기 전용)을 의미한다. 비워 둔다면 읽기 전용으로 가정한다.
file_for_reading1 = open('reading_file.txt', 'r')
file_for_reading2 = open('reading_file.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'reading_file.txt'

In [8]:
# 'w'는 wrtie(쓰기)를 의미한다.(해당 파일이 이미 존재한다면 기존 파일을 제거한다.)
file_for_writing = open('writing_file.txt', 'w')

In [10]:
# 'a'는 append(덧붙이기)를 의미한다(파일의 맨 끝에 덧붙인다.)
file_for_appending = open('appending_file.txt', 'a')

In [11]:
# 작업이 끝났다면 파일을 닫는 것을 잊지 말자.
file_for_writing.close()

파일을 닫는 것을 잊지 쉽기 때문에, 항상 작업이 끝나면 파일을 저절로 닫아주는 with안에서 파일 객체를 불러오자

In [12]:
with open(filename) as f:
    data = function_that_gets_data_from(f)


NameError: name 'filename' is not defined

In [13]:
# 이 시점부터는 f가 이미 종료되었기 때문에 f를 다시 사용하지 말자.
process(data)

NameError: name 'process' is not defined

만약 덱스트 파일 전체가 필요하다면 for를 사용해서 파일의 모든 줄을 반복해서 불러올 수 있다.

In [14]:
starts_with_hash = 0

with open('input.txt') as f:
    for line in f:  # 파일의 각 줄을 살펴본다.
        if re.match("^#", line):  # regex를 사용해서 줄이 "#"로 시작하는지 확인
            starts_with_hash += 1  # "#"로 시작한다면 1을 추가
            

FileNotFoundError: [Errno 2] No such file or directory: 'input.txt'

예를 들어 각 줄마다 이메일 주소 하나가 적혀 있는 파일을 사용해서 메일 도메인에 대한 히스토그램을 그려야 한다면, 도메인 이름은 꽤 정교한 규칙을 다르지만, @뒤에 오는 부분을 도메인으로 보는것은 꽤 괜찮은 결과를 준다.

In [15]:
def get_domain(email_address: str) -> str:
    """'@'기준으로 주소를 자르고 마지막 부분을 반환"""
    return email_address.lower().split("@")[-1]

# 몇 가지 테스트
assert get_domain('joelgrus@gmail.com') == "gmail.com"
assert get_domain('goel@m.datasciencester.com') == 'm.datasciencester.com'

In [16]:
from collections import Counter

with open('email_address.txt', 'r') as f:
    domain_counts = Counter(get_domain(line.strip()) for line in f if "@" in line)

FileNotFoundError: [Errno 2] No such file or directory: 'email_address.txt'

## 9.2.2 구분자가 있는 파일
대부분의 경우 데이터가 들어 있는 파일을 사용하게 될 것이다. 이 파일들은 보통 쉽표(,)나 탭(tab)으로 데이터의 시작과 끝이 구분되어 있다. \
하지만 쉼표, 탭, 개행 문자가 데이터 필드 자체에 포함되어 있으면 데이터 필드를 분리하는 것보다 파이썬에서 제공하는 csv 모듈이나 pandas 라이브러리를 사용하는 것을 추천 \
만약, 파일에 헤더가 없다면 csv.reader로 각 행을 리스트로 바꿀 수 있다. \
예를 들어 탭으로 분리된 주가 파일이 있다면 아래의 코드와 같이 처리 할 수 있다. (주가 파일은 날짜, 종목코드, 주가로 구성, tab으로 각 열 구분)

In [17]:
import csv

with open('tab_delimited_stock_prices.txt') as f:
    tab_reader = csv.reader(f, delimiter ='\t')
    for row in tab_reader:
        date = row[0]
        symbol = row[1]
        closing_price = float(row[2])
        process(date, symbol, closing_price)

FileNotFoundError: [Errno 2] No such file or directory: 'tab_delimited_stock_prices.txt'

만약 파일에 다음과 같이 헤더가 포함되어 있다면 date:symbol:closing_price \
첫 줄에서 reader.next를 사용해서 헤더 행을 건너뛰거나 csv.DictReader를 통해 헤더를 key로 사용하는 딕셔너리로 저장할 수 있다.

In [18]:
with open('colon_delimited_stock_prices.txt') as f:
    colon_reader = csv.DictReader(f, delimiter=":")
    for dict_row in colon_reader:
        date = dict_row["date"]
        symbol = dict_row["symbol"]
        closing_price = float(dict_row["closing_price"])
        process(date, symbol, closing_price)

FileNotFoundError: [Errno 2] No such file or directory: 'colon_delimited_stock_prices.txt'

파일의 헤더가 없더라도 filenames 파라미터로 key를 설정해서 DictReader를 사용할 수 있다. \
csv.writer를 사용해서 구분자가 있는 파일을 생성할 수도 있다.

In [19]:
todays_prices = {'AAPL': 90.91, 'MSFT': 41.68, 'FB': 64.5}

with open('comma_delimited_stock_prices.txt', 'w') as f:
    csv_writer = csv.writer(f, delimiter=',')
    for stock, price in todays_prices.items():
        csv_writer.writerow([stock, price])

데이터 필드 자체에 쉼표가 포함되어 있더라도 csv.writer는 올바른 방식으로 처리해 줄 것이다. 하지만 구분자가 있는 파일을 생성해 주는 코드를 직접 \
만들어 사용한다면 문제가 발생할 수도 있다. 예를 들어 아래와 같은 코드는